<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#构建初始化方法和select方法" data-toc-modified-id="构建初始化方法和select方法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>构建初始化方法和select方法</a></span></li><li><span><a href="#添加创建表格create方法" data-toc-modified-id="添加创建表格create方法-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>添加创建表格create方法</a></span></li><li><span><a href="#添加插入数据insert方法" data-toc-modified-id="添加插入数据insert方法-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>添加插入数据insert方法</a></span></li><li><span><a href="#添加删除数据drop和更新数据updata功能" data-toc-modified-id="添加删除数据drop和更新数据updata功能-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>添加删除数据drop和更新数据updata功能</a></span></li><li><span><a href="#完成代码" data-toc-modified-id="完成代码-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>完成代码</a></span></li></ul></div>

# 用class类对mysql功能进行封装

本课件继续修改思路:

将几个不同的功能的修改,变成继承的方式.

实践知识点: 方法重写.

## 构建初始化方法和select方法

mysql数据库的操作非常复杂,不易于使用.

设计一个Mysql数据库操作的类, 可以帮我们实现数据库操作的各种功能, 增删改查

我们直接用这个类,就可以方便的实现, 各种对数据库的操作

In [8]:
# 连接数据库的功能
class MysqlOperating:
    # 输出化方法, 当对象被实例化的时候, 就自动建立好和数据库的连接了
    def __init__(self , host ,user,password,database,charset,port = 3306):        
        import pymysql
        self.connect = pymysql.connect(host = host,
                         port=port,
                         user = user,
                         password= password,
                         database = database,
                         charset = charset)
        # 光标也直接构建好
        self.cursor = self.connect.cursor()
        print(self.cursor)
    def create(self, table,*args ):
        """创建表格功能
        table:表名
        *args: 每一个建立的字段及数据类型"""
        columns = ','.join(args)
        sql = "create table {} ({})".format(table,columns)
        print(sql)
        self.cursor.execute(sql)
        return '创建'+table+'成功'

In [9]:
mysql = MysqlOperating(host = "49.233.10.201", # 换成你自己的IP
                      user='cda0329',
                      password='123456',
                      database='pythongo',
                      charset='utf8',
                      port = 3306)

In [10]:
mysql.create('Student','id varchar(10)','name nvarchar(10)',
            'age int','sex nvarchar(10)')

create table Student (id varchar(10),name nvarchar(10),age int,sex nvarchar(10))


/Users/mac/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3720, 'NATIONAL/NCHAR/NVARCHAR implies the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using CHAR(x) CHARACTER SET UTF8MB4 in order to be unambiguous.')
  result = self._query(query)


'创建Student成功'

In [22]:
class MysqlOperatingV2(MysqlOperating):# 集成上一个版本的功能
    # 写一个插入功能
    def insert(self , table, data ):
        """插入数据
        table: 表名
        data: 数据二维表格式"""
        # %s 的个数有data里面有几列决定的
        n = len(data[0])
        s = ','.join(["%s" for i in range(n)])
        self.cursor.executemany("""insert into {} values({})""".format(table,s),data)
        # 提交
        self.connect.commit()

In [23]:
mysql = MysqlOperatingV2(host = "49.233.10.201", # 换成你自己的IP
                      user='cda0329',
                      password='123456',
                      database='pythongo',
                      charset='utf8',
                      port = 3306)

In [24]:
data = [['001','何炅','50','男'],
        ['002','撒贝宁','44','男'],
        ['003','王鸥','36','女'],
        ['004','鬼鬼','31','女'],
       ]

In [25]:
mysql.insert('Student',data)

In [26]:
mysql.create('Studentqq','id varchar(10)','name nvarchar(10)','age int','sex nvarchar(10)')

create table Studentqq (id varchar(10),name nvarchar(10),age int,sex nvarchar(10))


/Users/mac/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3720, 'NATIONAL/NCHAR/NVARCHAR implies the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using CHAR(x) CHARACTER SET UTF8MB4 in order to be unambiguous.')
  result = self._query(query)


'创建Studentqq成功'

In [5]:
class MysqlConnection:
    def __init__(self,ip,user,password,database,charset,port = 3306):
        import pymysql
        self.ip = ip
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.charset = charset
        self.connect = pymysql.connect(host=ip, 
                          port=port, 
                          user=user,
                          password=password, 
                          database=database,
                          charset=charset )
        self.cursor = self.connect.cursor()
    def select(self,col,table, condition=None):
        """查询条件"""
        sql = "select "+col+" from "+table
        # 判断是否有传递进来kwargs,也就是过滤条件
        if condition:
            sql+=" where " + condition
        # 执行代码
        self.cursor.execute(sql)
        self.data = self.cursor.fetchall()
        return self.data

In [6]:
mysql = MysqlConnection("117.51.137.136",'root','0123456789','pythongo',
                   charset='utf8',)

In [21]:
data = mysql.select('*','salgrade')
data

(('张三', 700, 1200),
 ('李四', 1201, 1400),
 ('王五', 1401, 2000),
 ('赵六', 2001, 3000),
 ('孙七', 3001, 9999))

In [9]:
data = mysql.select('name','salgrade','basic>1300')
data

(('王五',), ('赵六',), ('孙七',))

In [14]:
data = mysql.select('*','salgrade','basic<2500')
data

(('张三', 700, 1200), ('李四', 1201, 1400), ('王五', 1401, 2000), ('赵六', 2001, 3000))

In [22]:
data = mysql.select('*','salgrade','basic<2500 and basic > 1300')
data

(('王五', 1401, 2000), ('赵六', 2001, 3000))

In [24]:
mysql.select('*','salgrade','basic<2500 and basic > 1300')

(('王五', 1401, 2000), ('赵六', 2001, 3000))

In [25]:
mysql.data

(('王五', 1401, 2000), ('赵六', 2001, 3000))

## 添加创建表格create方法

In [35]:
class MysqlConnection:
    def __init__(self,ip,user,password,database,charset,port = 3306):
        import pymysql
        self.ip = ip
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.charset = charset
        self.connect = pymysql.connect(host=ip, 
                          port=port, 
                          user=user,
                          password=password, 
                          database=database,
                          charset=charset )
        self.cursor = self.connect.cursor()
    def select(self,col,table,condition=None):
        """查询条件"""
        sql = "select "+col+" from "+table
        # 判断是否有传递进来kwargs,也就是过滤条件
        if condition:
            sql+=" where " + condition
        # 执行代码
        self.cursor.execute(sql)
        self.data = self.cursor.fetchall()
        return self.data
    def create(self,table,*args):
        """创建表格"""
        # 创建表格
#         sql = "create table Student(Sid varchar(10),Sname nvarchar(10),Sage datetime,Ssex nvarchar(10))"
        columns = ','.join(args)
        print(columns)
        sql = "create table {} ({})".format(table,columns)
        print(sql)
        self.cursor.execute(sql)
        

In [36]:
mysql = MysqlConnection("117.51.137.136",'root','0123456789','pythongo',
                   charset='utf8',)

In [37]:
mysql.create('student2','sid varchar(10)','sname varchar(10)','sage datetime',
            'ssex nvarchar(10)')

sid varchar(10),sname varchar(10),sage datetime,ssex nvarchar(10)
create table student2 (sid varchar(10),sname varchar(10),sage datetime,ssex nvarchar(10))


/Users/mac/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3720, 'NATIONAL/NCHAR/NVARCHAR implies the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using CHAR(x) CHARACTER SET UTF8MB4 in order to be unambiguous.')
  result = self._query(query)


## 添加插入数据insert方法

In [55]:
class MysqlConnection:
    def __init__(self,ip,user,password,database,charset,port = 3306):
        import pymysql
        self.ip = ip
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.charset = charset
        self.connect = pymysql.connect(host=ip, 
                          port=port, 
                          user=user,
                          password=password, 
                          database=database,
                          charset=charset )
        self.cursor = self.connect.cursor()
    def select(self,col,table,condition=None):
        """查询条件"""
        sql = "select "+col+" from "+table
        # 判断是否有传递进来kwargs,也就是过滤条件
        if condition:
            sql+=" where " + condition
        # 执行代码
        self.cursor.execute(sql)
        self.data = self.cursor.fetchall()
        return self.data
    def create(self,table,*args):
        """创建表格"""
        # 创建表格
#         sql = "create table Student(Sid varchar(10),Sname nvarchar(10),Sage datetime,Ssex nvarchar(10))"
        columns = ','.join(args)
#         print(columns)
        sql = "create table {} ({})".format(table,columns)
        print(sql)
        self.cursor.execute(sql)
    
    def insert(self,table,data):
        """data: 二维表格形式"""
        self.cursor.executemany("""insert into {} values(%s, %s , %s , %s)""".format(table) , data)
        # 插入操作必须在commit后才会生效
        self.connect.commit()

In [56]:
mysql = MysqlConnection("117.51.137.136",'root','0123456789','pythongo',
                   charset='utf8',)

In [42]:
data = [['001','何炅','1970-2-3','男'],
        ['002','撒贝宁','1984-12-21','男'],
        ['003','王鸥','1986-4-4','女'],
        ['004','鬼鬼','1988-8-8','女'],
       ]

In [48]:
mysql.insert('student2',data)

In [50]:
mysql.create('users',
             'id varchar(10)',
             'username varchar(10)',
            'gender varchar(10)',
            'age int',
            )

create table users (id varchar(10),username varchar(10),gender varchar(10),age int)


In [57]:
data = [['001','何炅','男','42'],
        ['002','撒贝宁','男','39'],
        ['003','王鸥','女','32'],
        ['004','鬼鬼','女','30'],
       ]

In [58]:
mysql.insert('users',data)

In [59]:
mysql.select('*','users')

(('001', '何炅', '男', 42),
 ('002', '撒贝宁', '男', 39),
 ('003', '王鸥', '女', 32),
 ('004', '鬼鬼', '女', 30),
 ('001', '何炅', '男', 42),
 ('002', '撒贝宁', '男', 39),
 ('003', '王鸥', '女', 32),
 ('004', '鬼鬼', '女', 30))

## 添加删除数据drop和更新数据updata功能

In [7]:
class MysqlConnection:
    """数据库的增删改查功能整合"""
    def __init__(self,ip,user,password,database,charset,port = 3306):
        import pymysql
        self.ip = ip
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.charset = charset
        self.connect = pymysql.connect(host=ip, 
                          port=port, 
                          user=user,
                          password=password, 
                          database=database,
                          charset=charset )
        self.cursor = self.connect.cursor()
    def select(self,col,table,condition=None):
        """查询条件"""
        sql = "select "+col+" from "+table
        # 判断是否有传递进来kwargs,也就是过滤条件
        if condition:
            sql+=" where " + condition
        # 执行代码
        self.cursor.execute(sql)
        self.data = self.cursor.fetchall()
        return self.data
    def create(self,table,*args):
        """创建表格"""
        # 创建表格
#         sql = "create table Student(Sid varchar(10),Sname nvarchar(10),Sage datetime,Ssex nvarchar(10))"
        columns = ','.join(args)
#         print(columns)
        sql = "create table {} ({})".format(table,columns)
        print(sql)
        self.cursor.execute(sql)
    
    def insert(self,table,data):
        """data: 二维表格形式"""
        # 计算data中有几列
        n = len(data[0])
        # 拼接n个%s,
        s = ",".join(["%s" for i in range(n)])
        self.cursor.executemany("""insert into {} values({})""".format(table,s) , data)
        # 插入操作必须在commit后才会生效
        self.connect.commit()
    
    def delete(self, table ,condition=None):
        sql = "delete from "+table
        if condition:
            sql+= " where "+condition
#         print(sql)
        self.cursor.execute(sql)
        self.connect.commit() # 提交
        return '删除成功'
    def update(self,table,key,value,condition=None ):
        sql = "update {} set {} = '{}' where {}".format(table,key,value,condition)
        self.cursor.execute(sql)
        self.connect.commit() # 提交
        return '更改成功'

In [105]:
",".join(["%s" for i in range(5)])

'%s,%s,%s,%s,%s'

In [118]:
mysql = MysqlConnection("117.51.145.84",'cda','123456','pythongo',
                   charset='utf8',)

In [119]:
with open('student_grade.txt','r') as f:
    data = f.read()

In [120]:
data_split = [i.split('\t') for i in data.splitlines()]

In [121]:
data_split

[['姓名', '语文', '数学', '英语', '总分', '班名次'],
 ['杨璐', '131', '143', '144', '418', '1'],
 ['王雪', '131', '135', '144', '410', '2'],
 ['韩林霖', '127', '139', '142', '408', '3'],
 ['沙龙逸', '123', '148', '136', '407', '4'],
 ['李鉴学', '126', '135', '140', '401', '5'],
 ['韩雨萌', '129', '133', '138', '400', '6'],
 ['刘帅', '116', '143', '140', '399', '7'],
 ['康惠雯', '114', '142', '139', '395', '8'],
 ['刘钰婷', '115', '139', '135', '389', '9'],
 ['林世博', '116', '142', '129', '387', '10'],
 ['张希', '123', '130', '134', '387', '11'],
 ['徐冲', '122', '124', '139', '385', '12'],
 ['苑宇飞', '118', '136', '131', '385', '13'],
 ['卢一凡', '121', '123', '139', '383', '14'],
 ['张瑞鑫', '126', '115', '139', '380', '15'],
 ['范作鑫', '121', '127', '131', '379', '16'],
 ['裴子翔', '111', '139', '128', '378', '17'],
 ['武传禹', '119', '129', '130', '378', '18'],
 ['任雪桐', '124', '108', '144', '376', '19'],
 ['刘姗', '124', '128', '122', '374', '20'],
 ['王柏坤', '121', '123', '128', '372', '21'],
 ['赵永刚', '116', '131', '122', '369', '22'],
 ['张馨月大

In [122]:
# 添加一个新表
mysql.create('grade','name varchar(10)','Chinese int','Math int','English int',
            'score int','grade_rank int')

create table grade (name varchar(10),Chinese int,Math int,English int,score int,grade_rank int)


In [91]:
# 插入数据
# 准备数据
data_split

[['姓名', '语文', '数学', '英语', '总分', '班名次'],
 ['杨璐', '131', '143', '144', '418', '1'],
 ['王雪', '131', '135', '144', '410', '2'],
 ['韩林霖', '127', '139', '142', '408', '3'],
 ['沙龙逸', '123', '148', '136', '407', '4'],
 ['李鉴学', '126', '135', '140', '401', '5'],
 ['韩雨萌', '129', '133', '138', '400', '6'],
 ['刘帅', '116', '143', '140', '399', '7'],
 ['康惠雯', '114', '142', '139', '395', '8'],
 ['刘钰婷', '115', '139', '135', '389', '9'],
 ['林世博', '116', '142', '129', '387', '10'],
 ['张希', '123', '130', '134', '387', '11'],
 ['徐冲', '122', '124', '139', '385', '12'],
 ['苑宇飞', '118', '136', '131', '385', '13'],
 ['卢一凡', '121', '123', '139', '383', '14'],
 ['张瑞鑫', '126', '115', '139', '380', '15'],
 ['范作鑫', '121', '127', '131', '379', '16'],
 ['裴子翔', '111', '139', '128', '378', '17'],
 ['武传禹', '119', '129', '130', '378', '18'],
 ['任雪桐', '124', '108', '144', '376', '19'],
 ['刘姗', '124', '128', '122', '374', '20'],
 ['王柏坤', '121', '123', '128', '372', '21'],
 ['赵永刚', '116', '131', '122', '369', '22'],
 ['张馨月大

In [123]:
mysql.insert('grade',data_split[1:])

In [124]:
# 查询数据
data =mysql.select('*','grade','Math < 90 and Chinese <90 and English < 90')
data

(('张宇婷', 83, 61, 71, 215, 59),
 ('黄明涛', 87, 77, 43, 207, 60),
 ('吕文卓', 82, 53, 62, 197, 61),
 ('王晟煜', 79, 49, 64, 192, 62),
 ('满朝升', 78, 45, 47, 170, 65),
 ('李忠浩', 86, 32, 46, 164, 66),
 ('侯禹志', 75, 23, 34, 132, 67),
 ('尹鸿涛', 66, 23, 34, 123, 68))

In [125]:
# 修改数据
# 凡是数学不及格的,都给改成90分.
mysql.update('grade','Math','90', 'Math<90')

'更改成功'

In [126]:
# 删除数据, 凡是英语不及格的,都删除掉
mysql.delete('grade','English < 90')

'删除成功'

## 完成代码

In [40]:
# 连接数据库的功能
class MysqlOperating:
    # 输出化方法, 当对象被实例化的时候, 就自动建立好和数据库的连接了
    def __init__(self , host ,user,password,database,charset,port = 3306):        
        import pymysql
        self.connect = pymysql.connect(host = host,
                         port=port,
                         user = user,
                         password= password,
                         database = database,
                         charset = charset)
        # 光标也直接构建好
        self.cursor = self.connect.cursor()
        print(self.cursor)
    def create(self, table,*args ):
        """创建表格功能
        table:表名
        *args: 每一个建立的字段及数据类型"""
        columns = ','.join(args)
        sql = "create table {} ({})".format(table,columns)
        print(sql)
        self.cursor.execute(sql)
        return '创建'+table+'成功'
    # 写一个插入功能
    def insert(self , table, data ):
        """插入数据
        table: 表名
        data: 数据二维表格式"""
        # %s 的个数有data里面有几列决定的
        n = len(data[0])
        s = ','.join(["%s" for i in range(n)])
        self.cursor.executemany("""insert into {} values({})""".format(table,s),data)
        # 提交
        self.connect.commit()
        
    # 查询功能
    def select(self , col, table ,condition = None):
        """查询语句
        col : 列名
        table : 表名
        condition : 查询条件"""
        sql = 'select {} from {}'.format(col, table)
        # 判断是否传入了查询条件
        if condition:
            # 在刚才的sql语法后面添加过滤条件
            sql += " where "+ condition
        print(sql)
        # 执行查询
        self.cursor.execute(sql)
        # 提取数据,提取全部数据
        self.data = self.cursor.fetchall()
        return self.data
    # 删除数据    
    def delete(self, table ,condition=None):
        sql = "delete from "+table
        if condition:
            sql+= " where "+condition
        print(sql)
        self.cursor.execute(sql)
        self.connect.commit() # 提交
        return '删除成功'
    # 更新数据
    def update(self,table,key,value,condition=None ):
        sql = "update {} set {} = '{}' where {}".format(table,key,value,condition)
        self.cursor.execute(sql)
        self.connect.commit() # 提交
        return '更改成功'

In [41]:
mysql = MysqlOperating(host = "49.233.10.201", # 换成你自己的IP
                      user='cda0329',
                      password='123456',
                      database='pythongo',
                      charset='utf8',
                      port = 3306)

In [35]:
mysql.delete('Student','name = "王鸥"')

delete from Student where name = "王鸥"


'删除成功'

In [36]:
# 修改
mysql.update('Student','age','45','name="何炅"')

'更改成功'

In [29]:
mysql.select('*','Student')

select * from Student


(('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 31, '女'),
 ('001', '何炅', 50, '男'),
 ('002', '撒贝宁', 44, '男'),
 ('003', '王鸥', 36, '女'),
 ('004', '鬼鬼', 

In [30]:
mysql.select('name,age','Student')

select name,age from Student


(('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31),
 ('何炅', 50),
 ('撒贝宁', 44),
 ('王鸥', 36),
 ('鬼鬼', 31))

In [31]:
mysql.select('id,name,age','Student','sex = "女"')

select id,name,age from Student where sex = "女"


(('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31),
 ('003', '王鸥', 36),
 ('004', '鬼鬼', 31))

In [32]:
mysql.select('*','Student','name = "何炅"')

select * from Student where name = "何炅"


(('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'),
 ('001', '何炅', 50, '男'))

In [37]:
# 固定的代码
f = open('student_grade.txt','r')
data = f.read()
f.close()

In [38]:
# 建一个空列表
student_grade = []
for line in data.splitlines():
#     print(line)
    student_grade.append( line.split('\t') )

In [39]:
student_grade

[['姓名', '语文', '数学', '英语', '总分', '班名次'],
 ['杨璐', '131', '143', '144', '418', '1'],
 ['王雪', '131', '135', '144', '410', '2'],
 ['韩林霖', '127', '139', '142', '408', '3'],
 ['沙龙逸', '123', '148', '136', '407', '4'],
 ['李鉴学', '126', '135', '140', '401', '5'],
 ['韩雨萌', '129', '133', '138', '400', '6'],
 ['刘帅', '116', '143', '140', '399', '7'],
 ['康惠雯', '114', '142', '139', '395', '8'],
 ['刘钰婷', '115', '139', '135', '389', '9'],
 ['林世博', '116', '142', '129', '387', '10'],
 ['张希', '123', '130', '134', '387', '11'],
 ['徐冲', '122', '124', '139', '385', '12'],
 ['苑宇飞', '118', '136', '131', '385', '13'],
 ['卢一凡', '121', '123', '139', '383', '14'],
 ['张瑞鑫', '126', '115', '139', '380', '15'],
 ['范作鑫', '121', '127', '131', '379', '16'],
 ['裴子翔', '111', '139', '128', '378', '17'],
 ['武传禹', '119', '129', '130', '378', '18'],
 ['任雪桐', '124', '108', '144', '376', '19'],
 ['刘姗', '124', '128', '122', '374', '20'],
 ['王柏坤', '121', '123', '128', '372', '21'],
 ['赵永刚', '116', '131', '122', '369', '22'],
 ['张馨月大

In [42]:
# 对比
mysql = MysqlOperating(host = "49.233.10.201", # 换成你自己的IP
                      user='cda0329',
                      password='123456',
                      database='pythongo',
                      charset='utf8',
                      port = 3306)

In [44]:
mysql.create('grade','name varchar(10)','Chinese int',
            'Math int','English int', 'total_num int','rank1 int')

create table grade (name varchar(10),Chinese int,Math int,English int,total_num int,rank1 int)


'创建grade成功'

In [47]:
mysql.insert("grade",student_grade[1:])

In [48]:
# 英语不及格的同学
mysql.select('*','grade','English < 90')

select * from grade where English < 90


(('孙波', 96, 130, 89, 315, 43),
 ('洪禹', 106, 100, 80, 286, 52),
 ('王治超', 105, 95, 82, 282, 53),
 ('袁茂哲', 85, 95, 85, 265, 55),
 ('丁婉艺', 95, 71, 85, 251, 56),
 ('于娇莹', 97, 76, 71, 244, 57),
 ('刘旭辉', 99, 87, 44, 230, 58),
 ('张宇婷', 83, 61, 71, 215, 59),
 ('黄明涛', 87, 77, 43, 207, 60),
 ('吕文卓', 82, 53, 62, 197, 61),
 ('王晟煜', 79, 49, 64, 192, 62),
 ('孙龙基', 99, 21, 67, 187, 63),
 ('赵森', 90, 29, 64, 183, 64),
 ('满朝升', 78, 45, 47, 170, 65),
 ('李忠浩', 86, 32, 46, 164, 66),
 ('侯禹志', 75, 23, 34, 132, 67),
 ('尹鸿涛', 66, 23, 34, 123, 68))

In [49]:
mysql.delete('grade','Math >= 140')

delete from grade where Math >= 140


'删除成功'

In [50]:
mysql.update('grade', "Math", 90 ,"Math < 90" )

'更改成功'